In [1]:
import arrow
import numpy as np
import glob



# def get_windar(start,end):
#     start_run = arrow.get(start)
#     end_run = arrow.get(end)
#     thres = arrow.get('2014-09-12')

#     arrow_array = []
#     ncar = []

#     for r in arrow.Arrow.span_range('day', start_run, end_run):
#         arrow_array.append(r)

#     dayslen = len(arrow_array)
#     #print(dayslen)
#     ts = np.zeros(dayslen)
#     #print(arrow_array)

#     # ops_y2014m09d12.nc

#     for i in range(0,dayslen):

#         tdate = arrow_array[i][0]
#         yy = tdate.format('YYYY')
#         mm = tdate.format('MM')
#         dd = tdate.format('DD')
#         ymd = f'y{yy}m{mm}d{dd}'

#         if tdate < thres:
#             dirstring = '/results/forcing/atmospheric/GEM2.5/gemlam/'
#             guess = 'gemlam_'+ ymd +'.nc' 
#         else:
#             dirstring = '/results/forcing/atmospheric/GEM2.5/operational/'
#             guess = 'ops_'+ ymd +'.nc'

#         #tdir = '/data/tjarniko/results/RIV_PIL/BASE/ncs/'

#         w = glob.glob(dirstring+guess)
#         w = w[0]
#         ncar.append(w)
    
#     return ncar

# start ='2013-01-01'; end ='2013-12-31'
# ncar_2013 = get_windar(start,end)
# start ='2014-01-01'; end ='2014-12-31'
# ncar_2014 = get_windar(start,end)
# start ='2015-01-01'; end ='2015-12-31'
# ncar_2015 = get_windar(start,end)
# start ='2016-01-01'; end ='2016-12-31'
# ncar_2016 = get_windar(start,end)

In [2]:
start ='2013-01-01'; end ='2013-12-31'

def get_dates(start,end):
    start_run = arrow.get(start)
    end_run = arrow.get(end)
    thres = arrow.get('2014-09-12')

    arrow_array = []
    ncar = []

    for r in arrow.Arrow.span_range('day', start_run, end_run):
        arrow_array.append(r)

    dayslen = len(arrow_array)
    print(dayslen)
    #print(dayslen)
    ts = np.zeros(dayslen)
    #print(arrow_array)

    # ops_y2014m09d12.nc

    for i in range(0,dayslen):

        tdate = arrow_array[i][0]
        tdate = arrow_array[i][0]
        yy = tdate.format('YYYYMMDD')

        ncar.append(yy)
        
    return ncar
        
dar2013 = get_dates(start,end)


365


In [3]:
def weights_intrp_mtx_ops(odata):
    '''POINT: interpolates data from the atmospheric grid onto the ocean grid using a weights matrix | CALLS: | NOTES: this is much faster than interpolating with scipy! | TAKES : one array of size lats 266 x lons 256 of atmospheric data | GIVES: an array of NEMO-Salish size of the new data interpolated | USAGE: | ROLE IN CLUSTER PROJECT: | written by Michael Dunphy, adapted by tjšj, 2017'''
    from IPython import embed
    import netCDF4 as nc
    import scipy.interpolate as spi
    import scipy.sparse as sp
    import numpy as np
    '''takes a matrix of size lon(256), lat(266) and interpolates it onnto the NEMO gird
    size is 266 x 256'''
    weightsfile = '/home/mdunphy/MEOPAR/NEMO-forcing/grid/weights-gem2.5-ops.nc'
    with nc.Dataset(weightsfile) as f:
        s1 = f.variables['src01'][:]-1  # minus one for fortran-to-python indexing
        s2 = f.variables['src02'][:]-1
        s3 = f.variables['src03'][:]-1
        s4 = f.variables['src04'][:]-1
        w1 = f.variables['wgt01'][:]
        w2 = f.variables['wgt02'][:]
        w3 = f.variables['wgt03'][:]
        w4 = f.variables['wgt04'][:]
       
    NO = odata.size   # number of operational grid points
    NN = s1.size      # number of NEMO grid points
    
    # Build matrix
    n = np.array([x for x in range(0,NN)])
    M1 = sp.csr_matrix((w1.flatten(), (n, s1.flatten())), (NN,NO))
    M2 = sp.csr_matrix((w2.flatten(), (n, s2.flatten())), (NN,NO))
    M3 = sp.csr_matrix((w3.flatten(), (n, s3.flatten())), (NN,NO))
    M4 = sp.csr_matrix((w4.flatten(), (n, s4.flatten())), (NN,NO))
    M = M1+M2+M3+M4
    
        # Interpolate by matrix multiply - quite fast
    ndata = M*odata.flatten()

    # Reshape to NEMO shaped array
    ndata=ndata.reshape(s1.shape)
    
    return ndata

def weights_intrp_mtx_gemlam(odata):
    '''POINT: interpolates data from the atmospheric grid onto the ocean grid using a weights matrix | CALLS: | NOTES: this is much faster than interpolating with scipy! | TAKES : one array of size lats 266 x lons 256 of atmospheric data | GIVES: an array of NEMO-Salish size of the new data interpolated | USAGE: | ROLE IN CLUSTER PROJECT: | written by Michael Dunphy, adapted by tjšj, 2017'''
    from IPython import embed
    import netCDF4 as nc
    import scipy.interpolate as spi
    import scipy.sparse as sp
    import numpy as np
    '''takes a matrix of size lon(256), lat(266) and interpolates it onnto the NEMO gird
    size is 266 x 256'''
    weightsfile = '/data/tjarniko/MEOPAR/grid/weights-gem2.5-gemlam_201702.nc'
    with nc.Dataset(weightsfile) as f:
        s1 = f.variables['src01'][:]-1  # minus one for fortran-to-python indexing
        s2 = f.variables['src02'][:]-1
        s3 = f.variables['src03'][:]-1
        s4 = f.variables['src04'][:]-1
        w1 = f.variables['wgt01'][:]
        w2 = f.variables['wgt02'][:]
        w3 = f.variables['wgt03'][:]
        w4 = f.variables['wgt04'][:]
       
    NO = odata.size   # number of operational grid points
    NN = s1.size      # number of NEMO grid points
    
    # Build matrix
    n = np.array([x for x in range(0,NN)])
    M1 = sp.csr_matrix((w1.flatten(), (n, s1.flatten())), (NN,NO))
    M2 = sp.csr_matrix((w2.flatten(), (n, s2.flatten())), (NN,NO))
    M3 = sp.csr_matrix((w3.flatten(), (n, s3.flatten())), (NN,NO))
    M4 = sp.csr_matrix((w4.flatten(), (n, s4.flatten())), (NN,NO))
    M = M1+M2+M3+M4
    
        # Interpolate by matrix multiply - quite fast
    ndata = M*odata.flatten()

    # Reshape to NEMO shaped array
    ndata=ndata.reshape(s1.shape)
    
    return ndata

In [4]:
import arrow
import numpy as np
import glob



def get_windar(start,end):
    start_run = arrow.get(start)
    end_run = arrow.get(end)
    thres = arrow.get('2014-09-12')

    arrow_array = []
    ncar = []

    for r in arrow.Arrow.span_range('day', start_run, end_run):
        arrow_array.append(r)

    dayslen = len(arrow_array)
    #print(dayslen)
    ts = np.zeros(dayslen)
    #print(arrow_array)

    # ops_y2014m09d12.nc

    for i in range(0,dayslen):

        tdate = arrow_array[i][0]
        yy = tdate.format('YYYY')
        mm = tdate.format('MM')
        dd = tdate.format('DD')
        ymd = f'y{yy}m{mm}d{dd}'

        if tdate < thres:
            dirstring = '/results/forcing/atmospheric/GEM2.5/gemlam/'
            guess = 'gemlam_'+ ymd +'.nc' 
        else:
            dirstring = '/results/forcing/atmospheric/GEM2.5/operational/'
            guess = 'ops_'+ ymd +'.nc'

        #tdir = '/data/tjarniko/results/RIV_PIL/BASE/ncs/'

        w = glob.glob(dirstring+guess)
        w = w[0]
        ncar.append(w)
    
    return ncar

start ='2013-01-01'; end ='2013-12-31'
ncar_2013 = get_windar(start,end)
start ='2014-01-01'; end ='2014-12-31'
ncar_2014 = get_windar(start,end)
start ='2015-01-01'; end ='2015-12-31'
ncar_2015 = get_windar(start,end)
start ='2016-01-01'; end ='2016-12-31'
ncar_2016 = get_windar(start,end)

In [6]:
def produce_interpolated_wind_netcdf(start, end):
    '''POINT: produces a netcdf file containing SalishSeaNEMO sized arrays of wind magnitude, wind stress, and wind energy for each in a given series of days | CALLS: create_gemlist, weights_intrp_mtx (both from wind_fxn) | NOTES: day numbering starts at 0! the fname is hardcoded to a given directory into which nc files are deposited, could be changed maybe | TAKES : year, day start (starts at 0), day end  | GIVES: netcdfs in a given directory| USAGE: produce_interpolated_wind_netcdf | ROLE IN CLUSTER PROJECT: | written 2017, tjšj '''
    
    import netCDF4 as nc
    import xarray as xr
    
    ncar = get_windar(start,end)
    
    arrow_array = []
    start_run = arrow.get(start)
    end_run = arrow.get(end)
    for r in arrow.Arrow.span_range('day', start_run, end_run):
        arrow_array.append(r)
    
    for i in range(0, len(ncar)):
        if i%5 == 0:
            print(i)
        wind = nc.Dataset(ncar[i])
        #print(wind)
        u_wind = wind['u_wind'][:]
        v_wind = wind['v_wind'][:]

        wind_mag = np.sqrt((u_wind)*(u_wind)+(v_wind)*(v_wind))
        wind_stress = wind_mag ** 2
        wind_energy = wind_mag ** 3

        daily_wm = np.average(wind_mag, axis=0)
        daily_ws = np.average(wind_stress, axis=0)
        daily_we = np.average(wind_energy, axis=0)
        
        tdate = arrow_array[i][0]
        ymd = tdate.format('YYYYMMDD')
        
        thres = arrow.get('2014-09-12')
        if tdate<thres:
            daily_wm_int = weights_intrp_mtx_gemlam(daily_wm)
            daily_ws_int = weights_intrp_mtx_gemlam(daily_ws)
            daily_we_int = weights_intrp_mtx_gemlam(daily_we)
        
        else:
        
            daily_wm_int = weights_intrp_mtx_ops(daily_wm)
            daily_ws_int = weights_intrp_mtx_ops(daily_ws)
            daily_we_int = weights_intrp_mtx_ops(daily_we)  
            
        windthings = xr.Dataset({'daily_avg_windmag': (['y','x'], daily_wm_int),
                                 'daily_avg_windstress': (['y','x'], daily_ws_int),
                                 'daily_avg_windenergy': (['y','x'], daily_we_int),})
        
        fname = f'./WINDFILES_interp/windint_{ymd}.nc' 
        
        windthings.to_netcdf(fname)  
start ='2013-01-01'; end ='2013-12-31'
# produce_interpolated_wind_netcdf(start,end)

In [7]:
# start ='2014-01-01'; end ='2014-12-31'
# produce_interpolated_wind_netcdf(start,end)
# start ='2015-01-01'; end ='2015-12-31'
# produce_interpolated_wind_netcdf(start,end)
# start ='2016-01-01'; end ='2016-12-31'
# produce_interpolated_wind_netcdf(start,end)

In [8]:
def yearwinds_de(spacing,stn_b,stn_e,year):
    '''POINT: for a given set of stations, produces 3 timeseries of wind forcing data (magnitude, stress, energy)  station as lists in a single netcdf file (one netcdf file per station) | CALLS: import_bathy, make_stns, filster_station_in_domain (from map_fxn), reads in netcdf files produced by produce_interpolated_wind_netcdf | NOTES: days start at 0, stations start at 0, for a spacing of 10 there should be 580 stations, the start and end station option is so that it can be run in parallel if necessary, directory into which nc files are deposited is hardcoded under stn_name | TAKES : spacing (has been decided as 10 for this project), year, start station, end station | GIVES: netcdf files that contain 3 windrelated timeseries for each station, in a given repo | USAGE: yearwinds_de(10,1,580,2016) | ROLE IN CLUSTER PROJECT: | written 2017, tjšj '''
    import sys
    sys.path.append('./extraction_scripts')
    
    import map_fxn as mf
    import time
    import xarray as xr
    import numpy as np

    print('Spacing between stations: ' + str(spacing))
    
    noday = 365
    if year == 2016:
        noday = 366
    
    print('no. days, from yearwinds_de fxn')
    print(noday)
    
    bath = '/results/nowcast-sys/NEMO-forcing/grid/mesh_mask_SalishSea2.nc'
    grid = mf.import_bathy(bath)
    fmask = (grid.fmask[0,0,:,:])
    
    stn_x, stn_y = mf.make_stns(spacing)
    d_stn_x, d_stn_y = mf.filter_stn_in_domain(stn_x,stn_y,fmask)
    print('number of stns: ' + str(len(d_stn_x)))
    
    for stn in range(stn_b,stn_e):
    
        print('station is: ' ,str(stn))
        print('x is :', d_stn_x[stn])
        print('y is :', d_stn_y[stn])
        
        ts_x = d_stn_x[stn]
        ts_y = d_stn_y[stn]
        
        daily_windmag = np.zeros(noday)
        daily_windstress = np.zeros(noday)
        daily_windenergy = np.zeros(noday)
        
        end_day = noday
        
        start = f'{str(year)}-01-01'
        end = f'{str(year)}-12-31'

        start_run = arrow.get(start)
        end_run = arrow.get(end)

        arrow_array = []
        ncar = []

        for r in arrow.Arrow.span_range('day', start_run, end_run):
            arrow_array.append(r)
            
        

        dayslen = len(arrow_array)
        #print(dayslen)
        ts = np.zeros(dayslen)

        for day in range(0,end_day):

            
            tdate = arrow_array[day][0]
            yy = tdate.format('YYYYMMDD')
            filestart = f'./WINDFILES_interp/windint_{yy}.nc'

            wf = xr.open_dataset(filestart)
            ## extract wm, ws, we for the station for the day
            ## attach it to the corresponding array above
            ## save the resulting 3 lists to a netcdf file as below
            
            #avg_daily_windmag_windstress_energy
            
            daily_we = wf.daily_avg_windenergy[ts_y,ts_x]
            daily_we = daily_we.values
            daily_ws = wf.daily_avg_windstress[ts_y,ts_x]
            daily_ws = daily_ws.values
            daily_wm = wf.daily_avg_windmag[ts_y,ts_x]
            daily_wm = daily_wm.values
            ##be gentle about OB1 errors!!!!
            

            
            daily_windmag[day] = daily_wm
            daily_windstress[day] = daily_ws
            daily_windenergy[day] = daily_we
            
            
            winds = xr.Dataset({'wind_mags':(['t'], daily_windmag), 'wind_stresses':(['t'], daily_windstress), 
                                'wind_energy':(['t'], daily_windenergy)})
            stn_name = './NC_HINDCAST/' + str(year) + '/WIND_TS/stn_' + str(stn) + '_wind_data_sp' + str(spacing) + '.nc' 
            winds.to_netcdf(stn_name)
            

In [9]:
yearwinds_de(10,0,580,2014)

Spacing between stations: 10
no. days, from yearwinds_de fxn
365
number of stns: 580
station is:  0
x is : 2
y is : 392
station is:  1
x is : 2
y is : 402
station is:  2
x is : 2
y is : 412
station is:  3
x is : 2
y is : 422
station is:  4
x is : 2
y is : 432
station is:  5
x is : 2
y is : 442
station is:  6
x is : 2
y is : 452
station is:  7
x is : 2
y is : 462
station is:  8
x is : 12
y is : 382
station is:  9
x is : 12
y is : 392
station is:  10
x is : 12
y is : 402
station is:  11
x is : 12
y is : 412
station is:  12
x is : 12
y is : 422
station is:  13
x is : 12
y is : 432
station is:  14
x is : 12
y is : 442
station is:  15
x is : 22
y is : 372
station is:  16
x is : 22
y is : 382
station is:  17
x is : 22
y is : 392
station is:  18
x is : 22
y is : 402
station is:  19
x is : 22
y is : 412
station is:  20
x is : 22
y is : 422
station is:  21
x is : 32
y is : 352
station is:  22
x is : 32
y is : 362
station is:  23
x is : 32
y is : 372
station is:  24
x is : 32
y is : 382
station 

station is:  213
x is : 192
y is : 22
station is:  214
x is : 192
y is : 132
station is:  215
x is : 192
y is : 242
station is:  216
x is : 192
y is : 252
station is:  217
x is : 192
y is : 262
station is:  218
x is : 192
y is : 272
station is:  219
x is : 192
y is : 282
station is:  220
x is : 192
y is : 292
station is:  221
x is : 192
y is : 532
station is:  222
x is : 192
y is : 542
station is:  223
x is : 192
y is : 552
station is:  224
x is : 192
y is : 562
station is:  225
x is : 192
y is : 592
station is:  226
x is : 192
y is : 652
station is:  227
x is : 192
y is : 662
station is:  228
x is : 192
y is : 682
station is:  229
x is : 192
y is : 812
station is:  230
x is : 202
y is : 22
station is:  231
x is : 202
y is : 232
station is:  232
x is : 202
y is : 242
station is:  233
x is : 202
y is : 252
station is:  234
x is : 202
y is : 262
station is:  235
x is : 202
y is : 272
station is:  236
x is : 202
y is : 282
station is:  237
x is : 202
y is : 342
station is:  238
x is : 202

station is:  424
x is : 262
y is : 522
station is:  425
x is : 262
y is : 852
station is:  426
x is : 272
y is : 122
station is:  427
x is : 272
y is : 132
station is:  428
x is : 272
y is : 152
station is:  429
x is : 272
y is : 192
station is:  430
x is : 272
y is : 232
station is:  431
x is : 272
y is : 242
station is:  432
x is : 272
y is : 252
station is:  433
x is : 272
y is : 292
station is:  434
x is : 272
y is : 322
station is:  435
x is : 272
y is : 332
station is:  436
x is : 272
y is : 342
station is:  437
x is : 272
y is : 372
station is:  438
x is : 272
y is : 382
station is:  439
x is : 272
y is : 392
station is:  440
x is : 272
y is : 402
station is:  441
x is : 272
y is : 412
station is:  442
x is : 272
y is : 422
station is:  443
x is : 272
y is : 432
station is:  444
x is : 272
y is : 442
station is:  445
x is : 272
y is : 452
station is:  446
x is : 272
y is : 462
station is:  447
x is : 272
y is : 472
station is:  448
x is : 272
y is : 482
station is:  449
x is : 2

In [ ]:
w = nc.Dataset('/data/tjarniko/MEOPAR/analysis_tereza/notebooks/CLUSTER_PAPER/CLEAN/NC_hindcast/2013/WIND_TS/stn_0_wind_data_sp10.nc')